# Full CIFAR10 Dataset

In [1]:
# Full CIFAR10
import os.path

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms

In [2]:
## Hyperparameters.


# image channel 3=RGB, 1=Grayscale
img_channels = 3

# Class labels.
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = len(classes)

# Data directory.
data_dir = '../datasets'  # Dataset directory.
download = True           # Download dataset iff not already downloaded.
normalize = True          # Maybe normalize training data.

# Training parameters
batch_size = 16  # Mini-batch size.
lr = 1e-2        # Optimizer's learning rate.
epochs = 4       # Number of full passes over entire dataset.

In [3]:
# Should normalize images or not.
# Normalization helps convergence.
if normalize:
    # Transform rule: Convert to Tensor, Normalize images in range -1 to 1.
    transform = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.5, 0.5, 0.5), 
                                                         (0.5, 0.5, 0.5))])
else:
    # Transform rule: Convert to Tensor without normalizing image
    transform = transforms.Compose([transforms.ToTensor()])

# Download the training set and apply the transform rule to each.
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=download, transform=transform)
# Load the training set into mini-batches and shuffle them
trainset = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# Download the testing set and apply the transform rule to each.
testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=download, transform=transform)
# Load the testing set into mini-batches without shuffling.
testset = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Visualization function to visualize dataset.
def visualize(data, title='Visualization', smooth=False):
    # Iterate over the data.
    data_iter = iter(data)
    # Unpack images and labels.
    images, labels = data_iter.next()
    # Free up memory
    del data_iter
    # Join labels separated by spaces.
    labels = ' '.join(classes[label] for label in labels)
    # Call to helper function for plotting images.
    imshow(images, title=title, smooth=smooth)
    # Print out the labels.
    print('{} labels: {}'.format(title, labels))


# Helper function for plotting images.
def imshow(img, title, smooth=False):
    # Concatenate images into grids.
    img = torchvision.utils.make_grid(img)
    # Unnormalize image as necessary.
    img = img/2+0.5 if normalize else img
    # Interpolation parameters for image smoothening.
    smooth = 'bicubic' if smooth else 'spline16'
    # Convert the images to a numpy array.
    np_img = img.numpy()
    # Plot images
    plt.imshow(np.transpose(np_img, [1, 2, 0]), interpolation=smooth)
    plt.title(title)
    plt.show()


def plot_images(images, labels, pred=None, smooth=True):
    images = images / 2 + 0.5 if normalize else images
    # Create figure with sub-plots.
    fig, axes = plt.subplots(4, 4)

    # Adjust vertical spacing if we need to print ensemble and best-net.
    wspace, hspace = 0.3, 0.6 if pred else 0.3
    fig.subplots_adjust(hspace=hspace, wspace=wspace)

    for i, ax in enumerate(axes.flat):
        # Interpolation type.
        smooth = 'spline16' if smooth else 'nearest'

        # Plot image.
        ax.imshow(np.transpose(images[i], (1, 2, 0)), interpolation=smooth)
            
        # Name of the true class.
        labels_name = classes[labels[i]]

        # Show true and predicted classes.
        if pred is None:
            xlabel = f'True: {labels_name}'
        else:
            # Name of the predicted class.
            pred_name = classes[pred[i]]
            
            xlabel = f'True: {labels_name}\nPred: {pred_name}'

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()


# Let's visualize some training set.
# visualize(trainset, title='Training set', smooth=True)
train_iter = iter(trainset)
images, labels = train_iter.next()
plot_images(images, labels=labels, smooth=False)

In [ ]:
class Network(nn.Module):
    
    def __init__(self, **kwargs):
        super(Network, self).__init__()
        
        # Hyper-parameters
        self._img_channels = kwargs.get('img_channels')
        self._num_classes = kwargs.get('num_classes')
        
        # 2 convolutional & 3 fully connected layers
        self.conv1 = nn.Conv2d(self._img_channels, 16, 2)
        self.conv2 = nn.Conv2d(16, 32, 2)
        flatten_size = self.conv2.out_channels * 7 * 7
        self.fc1 = nn.Linear(flatten_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, self._num_classes)
    
    def forward(self, x):
        # Convolutional layers
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # Flatten layer
        x = x.view(-1, self._flatten(x))
        # Fully connected layers
        x = F.relu(self.fc1(x))     # relu + linear
        x = F.dropout(x, p=0.2)     # 20% dropout
        x = F.relu(self.fc2(x))     # relu + linear
        # Output layer
        x = F.sigmoid(self.fc3(x))  # sigmoid + linear
        return x
    
    def _flatten(self, x):
        size = x.size()[1:]  # input shape excluding batch dim.
        return torch.Tensor(size).numel()

In [ ]:
net = Network(img_channels=img_channels, num_classes=num_classes)

In [ ]:
# Loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

In [ ]:
# Training loop
for epoch in range(epochs):
    
    for i, data in enumerate(trainset):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        # Clear the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Forward + loss 
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()  # Back propagate
        
        # Updates learnable params
        optimizer.step()
        
        print(f'\rEpoch: {epoch+1:,}\tIter: {i+1:,}\tLoss: {loss.data[0]:.4f}', end='')
    print()
print('Finished training!')

In [ ]:
# Look at some test data
visualize(testset)

In [ ]:
# Let's make some predictions with the testset
test_iter = iter(testset)
images, labels = test_iter.next()

# Convert images to `autograd.Variable` before 
# passing through the network.
output = net(Variable(images))
print(output.data)

# Take the argmax of the predicted output
_, predictions = torch.max(output.data, dim=1)

# Visualize the predictions
imshow(images, title='Network predictions', smooth=True)
pred_labels = ' '.join(classes[pred] for pred in predictions)
print(f'Network prediction labels: {pred_labels}')

In [ ]:
# The performance for each class
correct_class = torch.zeros(len(classes))
total_class = torch.zeros(len(classes))

for data in testset:
    images, labels = data
    output = net(Variable(images))
    _, predictions = torch.max(output.data, dim=1)
    correct = (predictions == labels).squeeze()
    
    for i, label in enumerate(labels):
        correct_class += correct[i]
        total_class += 1

accuracy = correct_class / total_class
accuracy, _ = torch.sort(accuracy, descending=True)

for i, acc in enumerate(accuracy):
    print(f'Accuracy for {classes[i]} \t = {acc:.2%}')